In [1]:
from load_results import load_result_dataset
import pandas as pd

pn1 = 'full_fine_tuning_50epochs_edge_paper_final2'
pn2 = 'full_fine_tuning_50epochs_paper_final2'
pn3 = 'none'
final_data1 = load_result_dataset(pn1, pn2, pn3)
final_data1 = [{**d, 'ft_strategy': 'FFT (50 epochs)'} for d in final_data1]
df1 = pd.DataFrame(final_data1)

pn1 = 'full_fine_tuning_5epochs_edge_article1'
pn2 = 'full_fine_tuning_5epochs_article1'
pn3 = 'none'
final_data2 = load_result_dataset(pn1, pn2, pn3)
final_data2 = [{**d, 'ft_strategy': 'FFT (5 epochs)'} for d in final_data2]
df2 = pd.DataFrame(final_data2)

pn1 = 'linearprobe_50epochs_edge_paper_final2'
pn2 = 'linearprobe_50epochs_paper_final2'
pn3 = 'none'
final_data3 = load_result_dataset(pn1, pn2, pn3)
final_data3 = [{**d, 'ft_strategy': 'LP (50 epochs)'} for d in final_data3]
df3 = pd.DataFrame(final_data3)

df = pd.concat([ df1, df2, df3 ], axis=0, ignore_index=True) 

final_data = []
final_data.extend(final_data1)
final_data.extend(final_data2)
final_data.extend(final_data3)

../results/none/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/none/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
HEY
../results/full_fine_tuning_50

In [3]:
import pandas as pd
from scipy.stats import fisher_exact
from process_database import process_grouped_df, process_rankings
from load_results import load_result_dataset

# Collector for odds ratios and p-values
results = []
# group = 'model_type'
# group = 'pre_training_strategy'
group = 'model_size'

grouped_df = process_grouped_df(final_data)
grouped_df = process_rankings(grouped_df)
odds_df = grouped_df['TOTAL'].reset_index()
# Unique ID for observation
index_cols = ['backbone', 'pre_training_strategy', 'model_type', 'model_size', 'ft_strategy']
score_col = "score_sum"

for cond in set(odds_df[group] ):

        subset = odds_df[ odds_df[group] == cond ]

        pivot = subset.pivot_table(
                        index=index_cols,
                        columns="loss_function",
                        values=score_col ).dropna()
        
                # Pairwise TRADES vs CLASSIC
        pivot["winner"] = pivot.apply(
                        lambda row: "TRADES" if row["TRADES"] > row["Classic AT"]
                        else "CLASSIC" if row["TRADES"] < row["Classic AT"]
                        else "TIE",
                        axis=1
                        )
        
        # wins = pivot[pivot["winner"] != "TIE"]
        # trades_wins = (wins["winner"] == "TRADES").sum()
        # classic_wins = (wins["winner"] == "CLASSIC").sum()

        # table = [[trades_wins, classic_wins], [classic_wins, trades_wins]]
        #         # table = [[trades_wins, classic_wins], [classic_wins, trades_wins]]
        # odds_ratio, p_value = fisher_exact(table)

        # results.append({
        #         # "fts": fts,
        #         "cond": cond,
        #         "trades_wins": trades_wins,
        #         "classic_wins": classic_wins,
        #         "odds_ratio": odds_ratio,
        #         "p_value": p_value })
        
        
        for fts in [ "FFT (50 epochs)", "FFT (5 epochs)", "LP (50 epochs)" ]:
                 
                pivot2 = pivot.reset_index()
                pivot2 = pivot2[ pivot2.ft_strategy == fts ]

                wins = pivot2[pivot2["winner"] != "TIE"]
                trades_wins = (wins["winner"] == "TRADES").sum()
                classic_wins = (wins["winner"] == "CLASSIC").sum()

                table = [[trades_wins, classic_wins], [classic_wins, trades_wins]]
                # table = [[trades_wins, classic_wins], [classic_wins, trades_wins]]
                odds_ratio, p_value = fisher_exact(table)

                # Collect result
                results.append({
                                "fts": fts,
                                "cond": cond,
                                "trades_wins": trades_wins,
                                "classic_wins": classic_wins,
                                "odds_ratio": odds_ratio,
                                "p_value": p_value
                                })

# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df

Percentage of NaN values: 0.00%


,fts,cond,trades_wins,classic_wins,odds_ratio,p_value
0,FFT (50 epochs),small,5,2,6.250000,2.861305e-01
1,FFT (5 epochs),small,4,3,1.777778,1.000000e+00
2,LP (50 epochs),small,3,4,0.562500,1.000000e+00
3,FFT (50 epochs),large,18,0,inf,2.203824e-10
4,FFT (5 epochs),large,15,3,25.000000,1.519735e-04
5,LP (50 epochs),large,15,3,25.000000,1.519735e-04
6,FFT (50 epochs),medium,11,4,7.562500,2.683773e-02
7,FFT (5 epochs),medium,7,8,0.765625,1.000000e+00
8,LP (50 epochs),medium,10,5,4.000000,1.431110e-01


In [5]:
import plotly.express as px
import pandas as pd
import numpy as np

# Add sample size column
results_df["sample_size"] = results_df["trades_wins"] + results_df["classic_wins"]

# 3)  pretty text – use “∞” for the truly infinite ones
def fmt_or(x):
    if np.isinf(x):
        return "∞"
    return f"{x:.2f}" if x % 1 else f"{int(x)}"

results_df["text_label"] = (
    results_df.apply(
        lambda r: (
            f"{fmt_or(r['odds_ratio'])}"
            f"<br>(p={r['p_value']:.3f})"
            #f"<br>n={r['sample_size']}"
        ),
        axis=1,
    )
)

# cap infinite values at 10 % above the largest finite OR
finite_mask = np.isfinite(results_df["odds_ratio"])
finite_max  = results_df.loc[finite_mask, "odds_ratio"].max()
cap_value   = finite_max * 1.10

results_df["odds_ratio2"] = results_df["odds_ratio"].replace([np.inf, -np.inf],
                                                             cap_value)

if group == "pre_training_strategy":
    label_map = {
        "self-supervised (multimodal)": "self-sup<br>(multimodal)",
        "supervised (robust)": "sup<br>(robust)",
        "supervised": "sup",
        "hybrid": "hybrid",
        "self-supervised": "self-sup"
    }

    results_df["cond"] = results_df["cond"].map(label_map)


# Color-blind-friendly palette (Color Universal Design, up to 8 categories)
color_blind_palette = [
    "#E69F00",  # orange
    "#56B4E9",  # blue
    "#CC79A7",  # pink
    "#0072B2",  # dark blue
    "#D55E00",  # vermillion
    "#009E73",  # green
    "#F0E442",  # yellow
    "#999999",  # gray
]

# Optional: Create a mapping from unique `fts` values to color
fts_unique = results_df["fts"].unique()
color_discrete_map = {
    fts: color_blind_palette[i % len(color_blind_palette)]
    for i, fts in enumerate(fts_unique)
}


fig = px.bar(
    results_df,
    x="cond",
    y="odds_ratio2",
    color="fts",
    barmode="group",
    text="text_label",
    labels={"cond": "{}".format(group), "odds_ratio2": "Odds Ratio", "fts": "Fine-Tuning Strategy"},
    color_discrete_map=color_discrete_map
)

# Update traces for text styling
fig.update_traces(
    textposition="outside",
    textangle=-90,
    textfont=dict(size=6),
    cliponaxis=False
)

fig.update_layout(
            margin=dict(l=0, r=0, t=40, b=0),  # remove all outer margins
            width=450,    # in pixels (~2.5 inches at 96 DPI)
            height=400,    # in pixels (~1.875 inches)
            yaxis_title="Odds Ratio",
            xaxis_title="{}".format(group),            
            # White background
            plot_bgcolor="white",
            paper_bgcolor="white",
            
            # Light grey grid
            xaxis=dict(showgrid=False, gridcolor="lightgrey"),
            yaxis=dict(showgrid=True, gridcolor="lightgrey"),
            uniformtext_minsize=8,
            uniformtext_mode='show',
            showlegend = True,
            
            # Legend inside figure
            legend=dict(
                font=dict(size=12),
                yanchor="top",
                xanchor="left",   # anchor point for x position
                x=0.5,           # move legend to the left side
                y=1.1,           # optional: near the top
                bgcolor="rgba(255,255,255,1)",
                bordercolor="lightgrey",
                borderwidth=1
            )
        )

fig.show()
fig.write_image("./paper_figures/oddsratio_TRADES_outperforms_{}.png".format(group), scale=3  )
